# Assignment lab 03
- 29.03.21

## Master Class: Machine Learning (5MI2018)
- Faculty of Economic Science
- University of Neuchatel (Switzerland)
- Lab 3, see ML21_Exercise_3.pdf for more information

## Authors: 
- Romain Claret @RomainClaret
- Sylvain Robert-Nicoud @Nic0uds

In [1]:
# get the features names and the values of the categories from adult.names (build a dictionary)

data_dict = {}
with open('adult.names') as f:
    for l in f:
        if l[0] == '|' or ':' not in l: continue
        c = l.split(':')
        if c[1].startswith(' continuous'): data_dict[c[0]] = ""
        else: data_dict[c[0]] = c[1].replace("\n","").replace(".","").replace(" ","").split(",")

In [2]:
# in the specifications (adult.names): Unknown values are replaced with the character '?'

import pandas as pd

header = list(data_dict.keys())+['income']
df_train = pd.read_table("adult.data", sep=r',\s', na_values='?', header=None, names=header).dropna()
df_evaluate = pd.read_table("adult.test", sep=r',\s', na_values='?', skiprows=[0], header=None, names=header).dropna()

/usr/local/lib/python3.6/dist-packages/pandas/io/parsers.py:767: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return read_csv(**locals())
/usr/local/lib/python3.6/dist-packages/pandas/io/parsers.py:767: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return read_csv(**locals())


In [3]:
#import pandas_profiling
#df_train.profile_report()

In [4]:
#df_evaluate.profile_report()

In [5]:
# Standardizing of numeric values
# Doesn't have a lot of meaning in the case of decision trees as it's not using distances (like KNN)
# But it's just a pedagological flavor
# https://stats.stackexchange.com/questions/10289/whats-the-difference-between-normalization-and-standardization

#for c in df_train.select_dtypes(exclude=['object']):
#    df_train["stand_"+c] = df_train[c] - (df_train[c].mean() / df_train[c].std())
#    
#for c in df_evaluate.select_dtypes(exclude=['object']):
#    df_evaluate["stand_"+c] = df_evaluate[c] - (df_evaluate[c].mean() / df_evaluate[c].std())
#
#df_train.info(verbose=True)

In [6]:
# droping the education because it's redundant with education-num
# droping the occupation because it's not generic enough, we have much more categories that those captured in the training sample
# droping the relationship because it's not generic enough, we have much more categories that those captured in the training sample

drop_list = ["education", "occupation", "relationship"]
df_train = df_train.drop(columns=drop_list)
df_evaluate = df_evaluate.drop(columns=drop_list)

In [7]:
# reducing categories with multiple options into lower dimensions classification (into binary preferably) when possible
# - marital-status could be reduced as Married or Not-Married
# marital-status ['Never-married' 'Married-civ-spouse' 'Divorced' 'Married-spouse-absent' 'Separated' 'Married-AF-spouse' 'Widowed']
# - workclass could be recuded to 3 dimensions: Government, Private, and Self-Employment
# Note that we take into consideration all the options for the category from the specifications
# ['State-gov' 'Self-emp-not-inc' 'Private' 'Federal-gov' 'Local-gov' 'Self-emp-inc' 'Without-pay']

dict_replace = {
    'marital-status' : {
        'Never-married': 'Not-Married',
        'Married-civ-spouse': 'Married',
        'Divorced': 'Not-Married',
        'Married-spouse-absent': 'Married',
        'Separated': 'Married',
        'Married-AF-spouse': 'Married',
        'Widowed': 'Not-Married'
        },
    'workclass': {
        'State-gov': 'Government',
        'Self-emp-not-inc': 'Self-Employment',
        'Federal-gov': 'Government',
        'Local-gov': 'Government',
        'Self-emp-inc': 'Self-Employment'
        }
}

df_train.replace(dict_replace, inplace=True)
df_evaluate.replace(dict_replace, inplace=True)

In [8]:
# uniformizing the categories between the training and evaluation datasets
# indeed, there is a . at the end of the value in the evaluation dataset for the income category and not in the training dataset
df_evaluate["income"].replace({"<=50K.": "<=50K", ">50K.": ">50K"}, inplace=True)

In [9]:
from sklearn.preprocessing import LabelEncoder
# for binary categories we will be using a label encoder
# - marital-status, sex, income

for l in ["marital-status", "sex", "income"]:
    l_enc = LabelEncoder()
    encoder_train = l_enc.fit(df_train[l])
    encoder_evaluate = l_enc.fit(df_evaluate[l])
    df_train["encoded_"+l] = encoder_train.transform(df_train[l])
    df_evaluate["encoded_"+l] = encoder_evaluate.transform(df_evaluate[l])
    
#df_train.reset_index(inplace=True,drop=True)
#df_evaluate.reset_index(inplace=True,drop=True)

In [10]:
# For non-binary categories, first we check the specifications of the dataset to validate all the options per category (we have data_dict)
# Indeed, the values in the categories are not always all present in a dataset
# race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
# native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.
# and our custom category: workclass: Government, Private, and Self-Employment
# adding temporary fake data for the one hot encoder

fake_row = df_train[:1].copy()
df_fake = pd.DataFrame(data=fake_row, columns=df_train.columns)

cats_nonbinary = ["race", "native-country"]

for c in cats_nonbinary:
    for v in data_dict[c]:
        fake_row[c] = v
        df_fake = df_fake.append(fake_row, ignore_index=True)
        
cat_workclass = ["Government", "Private", "Self-Employment"]
for cw in cat_workclass:
    fake_row["workclass"] = cw
    df_fake = df_fake.append(fake_row, ignore_index=True)
    
df_train = df_train.append(df_fake).reset_index(drop=True)
df_evaluate = df_evaluate.append(df_fake).reset_index(drop=True)

In [11]:
#not_columns = ['encoded_income',"encoded_sex", "encoded_marital-status", "income"]
#df_train.loc[:, df_train.columns != 'encoded_income']
#df_train[map(lambda x :x not in not_columns, list(df_train.columns))]
#df_train[df_train.columns.difference(not_columns)]

In [12]:
# get meaningful columns
continuous_features = [k for k, v in data_dict.items() if v == ""]
unencoded_features = ["workclass", "race", "native-country"]
encoded_features = [c for c in df_train if c.startswith('encoded')]
columns = continuous_features+unencoded_features+encoded_features
#columns.remove("encoded_income")
columns

['age',
 'fnlwgt',
 'education-num',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'workclass',
 'race',
 'native-country',
 'encoded_marital-status',
 'encoded_sex',
 'encoded_income']

In [13]:
#encoded_features[:-1]

In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

# Standardizing of numeric values
# Doesn't have a lot of meaning in the case of decision trees as it's not using distances (like KNN)
# But it's just a pedagological flavor and to use the ColumnTranformer for whatever reason 
# https://stats.stackexchange.com/questions/10289/whats-the-difference-between-normalization-and-standardization

# We choose a the best parameters from lab2 for the decision tree
# depth=8 Train accuracy_score 0.8550292179535
# depth=8 Test accuracy_score 0.8465108569534229
# depth=8 Evaluation accuracy_score 0.8469455511288181 

feature_transformation = ColumnTransformer(transformers=[
    ('categorical', OneHotEncoder(handle_unknown='ignore'), unencoded_features+encoded_features[:-1]),
    ('numerical', StandardScaler(), continuous_features)
])

adult_pipeline = Pipeline(steps=[
  ('features', feature_transformation),
  ('classifier', DecisionTreeClassifier(criterion='gini', random_state=1, max_depth=8))
])

In [15]:
# Building our pipeline-driven model
pipeline_model = adult_pipeline.fit(df_train, df_train["encoded_income"])

In [16]:
pipeline_model

Pipeline(steps=[('features',
                 ColumnTransformer(transformers=[('categorical',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['workclass', 'race',
                                                   'native-country',
                                                   'encoded_marital-status',
                                                   'encoded_sex']),
                                                 ('numerical', StandardScaler(),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week'])])),
                ('classifier',
                 DecisionTreeClassifier(max_depth=8, random_state=1))])

In [17]:
#y_pred = pipeline_model.predict(df_evaluate)
#y_pred

In [18]:
# for non-binary categories we will be using a onehot encoder as decision trees are sensitive to leaves values
# note that get_dummies from pandas is exactly doing this without the complexity of using OneHotEncoder manually from sklearn
# - workclass, race, native-country

#for l in ["workclass", "race", "native-country"]:
#    df_train=pd.concat([df_train,pd.get_dummies(df_train[l], prefix="encoded_"+l)],axis=1)
#    df_evaluate=pd.concat([df_evaluate,pd.get_dummies(df_evaluate[l], prefix="encoded_"+l)],axis=1)
#

In [19]:
#remove the fake rows
df_train = df_train[:-len(df_fake)]
df_evaluate = df_evaluate[:-len(df_fake)]

In [20]:
# get meaningful columns
#continuous_features = [k for k, v in data_dict.items() if v == ""]
#encoded_features = [c for c in df_train if c.startswith('encoded')]
#columns = continuous_features+encoded_features
#columns.remove("encoded_income")

In [21]:
## Converting objects columns into values
#for c in continuous_features+["encoded_income"]:
#    df_train[c] = df_train[c].astype('int64')
#    df_evaluate[c] = df_evaluate[c].astype('int64')
#
#for c in [c for c in df_train.select_dtypes(['object']) if c.startswith('encoded')]:
#    df_train[c] = df_train[c].astype('uint8')
#    df_evaluate[c] = df_evaluate[c].astype('uint8')
#
#df_train.info()

In [22]:
#from sklearn.model_selection import train_test_split
#
## make training and testings sets
#X_train, X_test, y_train, y_test = train_test_split(df_train[columns],df_train["encoded_income"],test_size=0.2,random_state=1)
#
## make evaluation sets
#X_evaluate = df_evaluate[columns]
#y_evaluate = df_evaluate["encoded_income"]

In [23]:
from sklearn.model_selection import train_test_split

# make training and testings sets
X_train, X_test, y_train, y_test = train_test_split(df_train,df_train["encoded_income"],test_size=0.2,random_state=1)

# make evaluation sets
X_evaluate = df_evaluate
y_evaluate = df_evaluate["encoded_income"]

In [24]:
#df_evaluate.shape

In [25]:
# run a decision tree classifier

#from sklearn.tree import DecisionTreeClassifier
#from sklearn.metrics import accuracy_score
#import matplotlib.pyplot as plt
#
#parameter_dtree_min = 1
#parameter_dtree_max = 15
#preds_dtree_train=[]
#preds_dtree_test=[]
#for depth in range(parameter_dtree_min,parameter_dtree_max):
#    cl_dtree = DecisionTreeClassifier(criterion='gini', random_state=1,max_depth=depth)
#    dtree_model = cl_dtree.fit(X_train,y_train)
#    y_hat_dtree_train = dtree_model.predict(X_train)
#    y_hat_dtree_test = dtree_model.predict(X_test)
#    preds_dtree_train.append(accuracy_score(y_train,y_hat_dtree_train))
#    preds_dtree_test.append(accuracy_score(y_test,y_hat_dtree_test))
#    #print(depth,"Train accuracy_score",preds_train[-1])
#    #print(depth,"Test accuracy_score",preds_test[-1],"\n")
#    
#plt.scatter(range(parameter_dtree_min,parameter_dtree_max),preds_dtree_train,c="b",label="train score")
#plt.scatter(range(parameter_dtree_min,parameter_dtree_max),preds_dtree_test,c="r",label="test score")
#plt.legend(loc="upper left")
#plt.title('DecisionTreeClassifier: accuracy_score vs depth')
#plt.xlabel('depth')
#plt.ylabel('accuracy_score')
#plt.savefig('DecisionTreeClassifier.png')
#plt.show()

In [26]:
#present depth with best score for evaluation dataset
from sklearn.metrics import accuracy_score

y_hat_dtree_train = pipeline_model.predict(X_train)
y_hat_dtree_test = pipeline_model.predict(X_test)
y_hat_dtree_evaluate = pipeline_model.predict(X_evaluate)

print("Train accuracy_score",accuracy_score(y_train,y_hat_dtree_train))
print("Test accuracy_score",accuracy_score(y_test,y_hat_dtree_test))
print("Evaluation accuracy_score",accuracy_score(y_evaluate,y_hat_dtree_evaluate),"\n")

Train accuracy_score 0.8539931203116582
Test accuracy_score 0.8518150174042765
Evaluation accuracy_score 0.8470783532536521 



In [27]:
import pickle
pickle.dump(pipeline_model, open("pipeline_model.pickle", "wb" ))

In [29]:
filename = 'adult.test'
file_handler = open(filename, 'r').readlines()
prefix_file = "adult_2021_cw_"
week_number = 1
split_into = 10
line_count = 0
with open(filename) as f: line_count = sum(1 for _ in f)

for i in range(len(file_handler)):
    if i % (line_count//split_into) == 0:
        open(str(prefix_file)+str(week_number) + ".csv", "w+").writelines(file_handler[i:i+1000])
        week_number += 1

In [ ]:
import pickle
pipeline_model = pickle.load( open("pipeline_model.pickle", "rb" ))

In [30]:
import warnings
warnings.filterwarnings("ignore")

weeks_count = 10
filename = 'adult.test'
prefix_file = "adult_2021_cw_"

# get the features names and the values of the categories from adult.names (build a dictionary)

data_dict = {}
with open('adult.names') as f:
    for l in f:
        if l[0] == '|' or ':' not in l: continue
        c = l.split(':')
        if c[1].startswith(' continuous'): data_dict[c[0]] = ""
        else: data_dict[c[0]] = c[1].replace("\n","").replace(".","").replace(" ","").split(",")
            
header = list(data_dict.keys())+['income']
            
for i in range (weeks_count):
    filename = str(prefix_file)+str(i+1)+".csv"
    df_weekly = pd.read_table(filename, sep=r',\s', na_values='?', skiprows=[0], header=None, names=header).dropna()
    
    drop_list = ["education", "occupation", "relationship"]
    df_weekly = df_weekly.drop(columns=drop_list)
    
    dict_replace = {
    'marital-status' : {
        'Never-married': 'Not-Married',
        'Married-civ-spouse': 'Married',
        'Divorced': 'Not-Married',
        'Married-spouse-absent': 'Married',
        'Separated': 'Married',
        'Married-AF-spouse': 'Married',
        'Widowed': 'Not-Married'
        },
    'workclass': {
        'State-gov': 'Government',
        'Self-emp-not-inc': 'Self-Employment',
        'Federal-gov': 'Government',
        'Local-gov': 'Government',
        'Self-emp-inc': 'Self-Employment'
        }
    }

    df_weekly.replace(dict_replace, inplace=True)
    
    df_weekly["income"].replace({"<=50K.": "<=50K", ">50K.": ">50K"}, inplace=True)
    
    for l in ["marital-status", "sex", "income"]:
        l_enc = LabelEncoder()
        encoder_weekly = l_enc.fit(df_weekly[l])
        df_weekly["encoded_"+l] = encoder_weekly.transform(df_weekly[l])
    
    y_hat_dtree_weekly = pipeline_model.predict(df_weekly)

    print(filename, "accuracy_score:",accuracy_score(df_weekly["encoded_income"],y_hat_dtree_weekly),"\n")
    
    pd.DataFrame(y_hat_dtree_weekly).to_csv(str(prefix_file)+str(i+1)+"_pred.csv",header=["pred_income"], index=None)

adult_2021_cw_1.csv accuracy_score: 0.8293736501079914 

adult_2021_cw_2.csv accuracy_score: 0.8503253796095445 

adult_2021_cw_3.csv accuracy_score: 0.8427807486631016 

adult_2021_cw_4.csv accuracy_score: 0.8307860262008734 

adult_2021_cw_5.csv accuracy_score: 0.8507462686567164 

adult_2021_cw_6.csv accuracy_score: 0.854978354978355 

adult_2021_cw_7.csv accuracy_score: 0.8545454545454545 

adult_2021_cw_8.csv accuracy_score: 0.8514531754574811 

adult_2021_cw_9.csv accuracy_score: 0.8296943231441049 

adult_2021_cw_10.csv accuracy_score: 0.8574537540805223 



In [31]:
# lab 01 results:
# depth=8 Train accuracy_score 0.8356749140038957
# depth=8 Test accuracy_score 0.832587435769932
# depth=8 Evaluation accuracy_score 0.8287516600265604 

In [1]:
%run main.py

/usr/local/lib/python3.6/dist-packages/pandas/io/parsers.py:767: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return read_csv(**locals())
/usr/local/lib/python3.6/dist-packages/pandas/io/parsers.py:767: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return read_csv(**locals())


******************************************************
Evaluate Decision Tree Classifier Pipeline on new data
****************************************************** 

Train accuracy_score 0.8539931203116582
Test accuracy_score 0.8518150174042765
Evaluation accuracy_score 0.8470783532536521 



In [3]:
%run predict_income.py

adult_2021_cw_1_pred.csv accuracy_score: 0.8293736501079914 

adult_2021_cw_2_pred.csv accuracy_score: 0.8503253796095445 

adult_2021_cw_3_pred.csv accuracy_score: 0.8427807486631016 

adult_2021_cw_4_pred.csv accuracy_score: 0.8307860262008734 

adult_2021_cw_5_pred.csv accuracy_score: 0.8507462686567164 

adult_2021_cw_6_pred.csv accuracy_score: 0.854978354978355 

adult_2021_cw_7_pred.csv accuracy_score: 0.8545454545454545 

adult_2021_cw_8_pred.csv accuracy_score: 0.8514531754574811 

adult_2021_cw_9_pred.csv accuracy_score: 0.8296943231441049 

adult_2021_cw_10_pred.csv accuracy_score: 0.8574537540805223 

